In [1]:
import logging

import numpy as np
import optimistix as optx
from atmodeller import (
    InteriorAtmosphere,
    Planet,
    Species,
    SpeciesCollection,
    debug_logger,
    earth_oceans_to_hydrogen_mass,
    SolverParameters,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm

from scipy.interpolate import interp1d
import os

from matplotlib.ticker import ScalarFormatter

formatter = ScalarFormatter(useMathText=False)
formatter.set_scientific(False)
formatter.set_useOffset(False)
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

logger = debug_logger()
logger.setLevel(logging.INFO)

# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

Atmodeller initialized with double precision (float64)


In [2]:
number_of_realisations = 50
surface_temperature = 2000  # K -> take another temperature, the one aligning with mbe, up to 2-3k kelvin https://link.springer.com/article/10.1007/s11214-020-00660-3
# For simulations with fixed mass and surface radius:
MEB_mass = 1e25 #1-2 earth masses for inside boundary (90% somewhat, could be less, reask) https://iopscience.iop.org/article/10.3847/PSJ/ad3b93/pdf
#planet_mass = 8.68e25 
planet_mass = 8.68e25 
surface_radius = 2.5559e7  # Lindal et al -> Can be discussed, radius of magma envelope boundary -> reduce and deal with incostitent between rocky and ice planets
#will iterate over later, take for now 0.8 r_U (I find results of 0.4-0.9). r_U = 2.5559e7 https://link.springer.com/article/10.1007/s11214-020-00660-3 
#radius of rocky part -> what does planet class do for surface radius 
mantle_melt_fraction = 1.0  # 0.1 -> we will iterate 

RANDOM_SEED = 0

np.random.seed(RANDOM_SEED)

In [3]:
H_masspercent_lodders09: float = 73.9
He_masspercent_lodders09: float = 24.69
C_masspercent_lodders09: float = 0.22
N_masspercent_lodders09: float = 0.07
O_masspercent_lodders09: float = 0.63
Si_masspercent_lodders09: float = 0.07
Mg_masspercent_lodders09: float = 0.06

H_logN: float = 12
He_logN: float = 10.93
C_logN: float = 8.39
N_logN: float = 7.86
O_logN: float = 8.73
Si_logN: float = 7.53
Mg_logN: float = 7.54 

In [4]:
#SiO2 for primitive composition of Earth
SiO2_mantlemasspercent_palme14: float = 45.4
total_mantlemasspercent_palme14: float = 98.41
core_mass_fraction: float = 0.327  # typical values used are between 0.325 - 0.33
Si_massfraction_palme14: float = round(
    SiO2_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction)
    / (28.0855 + 2 * 15.999)
    * 28.0855,
    3,
)

O_massfraction_palme141: float = Si_massfraction_palme14 / 28.0855 * 2 * 15.999

MgO_mantlemasspercent_palme14: float = 36.77
Mg_massfraction_palme14: float = round(
    MgO_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction)
    / (24.305 + 15.999)
    * 24.305,
    3,
)

O_massfraction_palme142: float = Si_massfraction_palme14 / 24.305 * 15.999
O_massfraction_palme14 = O_massfraction_palme141 + O_massfraction_palme142

#envelope mass accreted (not final)
f_envs = np.logspace(-3, -1, num=number_of_realisations)
envelope_mass = f_envs * planet_mass
#mantle_mass = (1 - f_envs) * planet_mass

si_kg_magma: float = Si_massfraction_palme14 * planet_mass
mg_kg_magma: float = Mg_massfraction_palme14 * planet_mass
o_kg_magma: float = O_massfraction_palme14 * planet_mass

# Lodders et al. (2009) Springer book chapter Table 8
h_kgs = envelope_mass * H_masspercent_lodders09 / 100
si_kgs_solar = h_kgs * Si_masspercent_lodders09 / H_masspercent_lodders09
mg_kgs_solar = h_kgs * Mg_masspercent_lodders09 / H_masspercent_lodders09
o_kgs_solar = h_kgs * O_masspercent_lodders09 / H_masspercent_lodders09
c_kgs_solar = h_kgs * C_masspercent_lodders09 / H_masspercent_lodders09
n_kgs_solar = h_kgs * N_masspercent_lodders09 / H_masspercent_lodders09
he_kgs_solar = h_kgs * He_masspercent_lodders09 / H_masspercent_lodders09

In [5]:
eos_models = get_eos_models()
sol_models = get_solubility_models()

H2O_g = Species.create_gas("H2O")
H2O_gs = Species.create_gas("H2O", solubility=sol_models["H2O_peridotite_sossi23"])
H2O_rgs = Species.create_gas(
    "H2O",
    activity=eos_models["H2O_cork_holland98"],
    solubility=sol_models["H2O_peridotite_sossi23"],
)

H2_g = Species.create_gas("H2")
H2_gs = Species.create_gas("H2", solubility=sol_models["H2_basalt_hirschmann12"])
H2_rgs = Species.create_gas(
    "H2", activity=eos_models["H2_chabrier21"], solubility=sol_models["H2_basalt_hirschmann12"]
)

O2_g = Species.create_gas("O2")
O2_rg = Species.create_gas("O2", activity=eos_models["O2_cs_shi92"])

OSi_g = Species.create_gas("OSi")
OSi_rg = Species.create_gas("OSi", activity=eos_models["OSi_rk49_connolly16"])

H4Si_g = Species.create_gas("H4Si")
H4Si_rg = Species.create_gas("H4Si", activity=eos_models["H4Si_wang18"])

O2Si_l = Species.create_condensed("O2Si", state="l")
O2Si_bqz = Species.create_condensed("O2Si", state="bqz")

CO2_g = Species.create_gas("CO2")
CO2_gs = Species.create_gas("CO2", solubility=sol_models["CO2_basalt_dixon95"])
CO2_rgs = Species.create_gas(
    "CO2", solubility=sol_models["CO2_basalt_dixon95"], activity=eos_models["CO2_cs_shi92"]
)

CO_g = Species.create_gas("CO")
CO_gs = Species.create_gas("CO", solubility=sol_models["CO_basalt_yoshioka19"])
CO_rgs = Species.create_gas(
    "CO", solubility=sol_models["CO_basalt_yoshioka19"], activity=eos_models["CO_cs_shi92"]
)

CH4_g = Species.create_gas("CH4")
CH4_gs = Species.create_gas("CH4", solubility=sol_models["CH4_basalt_ardia13"])
CH4_rgs = Species.create_gas(
    "CH4", solubility=sol_models["CH4_basalt_ardia13"], activity=eos_models["CH4_cs_shi92"]
)

N2_g = Species.create_gas("N2")
N2_gs = Species.create_gas("N2", solubility=sol_models["N2_basalt_libourel03"])
N2_rgs = Species.create_gas(
    "N2", solubility=sol_models["N2_basalt_libourel03"], activity=eos_models["N2_cs_saxena87"]
)

NH3_g = Species.create_gas("NH3")
NH3_rg = Species.create_gas("H3N", activity=eos_models["H3N_rk49_reid87"])

He_g = Species.create_gas("He")
He_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
He_rgs = Species.create_gas(
    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
)

Mg_g = Species.create_gas("Mg")
#Mg_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
#Mg_rgs = Species.create_gas(
#    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
#)

MgO_l = Species.create_condensed("MgO", state="l")
MgO_cr = Species.create_condensed("MgO", state="cr")

MgH_g = Species.create_gas("MgH")

#Let's wait for this, first the more common
#MgH2_b = Species.create_condensed("MgH2", state="b")
#MgH2_l = Species.create_condensed("MgH2", state="l")

MgOH_g = Species.create_gas("MgOH")

species_HHeCNOSi_magma_nosol_ideal = SpeciesCollection(
    (H2O_g, H2_g, O2_g, OSi_g, H4Si_g, O2Si_l, CO2_g, CO_g, CH4_g, N2_g, NH3_g, He_g, Mg_g, MgO_l, MgO_cr, MgH_g, MgOH_g)
)
species_HHeCNOSi_magma_sol_ideal = SpeciesCollection(
    (H2O_gs, H2_gs, O2_g, OSi_g, H4Si_g, O2Si_l, CO2_gs, CO_gs, CH4_gs, N2_gs, NH3_g, He_gs, Mg_g, MgO_l, MgO_cr, MgH_g, MgOH_g)
)
species_HHeCNOSi_magma_sol_real = SpeciesCollection(
    (
        H2O_rgs,
        H2_rgs,
        O2_rg,
        OSi_rg,
        H4Si_rg,
        O2Si_l,
        CO2_rgs,
        CO_rgs,
        CH4_rgs,
        N2_rgs,
        NH3_rg,
        He_rgs,
        Mg_g, 
        MgO_l,
        MgO_cr, 
        MgH_g,  
        MgOH_g
    )
)
#MgH Magnesium Hydride
#MgOH Magnesium Hydroxide -> ideal gas law 
#Plot condensed moles for example for Silicon
#mass balance, take mass constraints, take atmospheric mass/moles+ take condensed mass/moles + take dissolved mass/moles
#plot envelope mass fraction, envelope mass/planet mass, Helled, what do they call the volatile layer. 
#temperature-pressure profile at 1 Myr or 10 Myr
#Ravit Helled 2011-2013

In [6]:
color_tot = "black"
color_H2 = "orange"
color_H2O = "blue"
color_SiH4 = "red"
color_SiO = "brown"
color_O2 = "limegreen"
color_CO2 = "cyan"
color_CO = "magenta"
color_CH4 = "purple"
color_N2 = "green"
color_NH3 = "pink"
color_MgO = "darkred"
color_He = "gray"
color_H = "orange"
color_C = "cyan"
color_N = "green"
color_O = "blue"
color_Si = "red"
color_Mg = "lime"
color_MgH2 = "gold"
color_MgH = "teal"
color_MgOH = "indigo"

In [7]:
def create_models(surface_temperature, planet_mass, mantle_melt_fraction, surface_radius, mass_constraints):
    
    Uranus = Planet(
            surface_temperature=surface_temperature,
            planet_mass=planet_mass,
            mantle_melt_fraction=mantle_melt_fraction,
            surface_radius=surface_radius,
        )

    model_magma_nosol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_nosol_ideal)
    model_magma_nosol_ideal.solve(
            planet=Uranus,
            mass_constraints=mass_constraints,
            )
    output_magma_nosol_ideal = model_magma_nosol_ideal.output
    output_magma_nosol_ideal.quick_look()
    output_magma_nosol_ideal.to_excel("HHeCNOSi_magma_nosol_ideal")

    model_magma_sol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_sol_ideal)
    initial_log_number_density = output_magma_nosol_ideal.log_number_density
    model_magma_sol_ideal.solve(
            planet=Uranus,
            mass_constraints=mass_constraints,
            initial_log_number_density=initial_log_number_density,
            )
    output_magma_sol_ideal = model_magma_sol_ideal.output
    output_magma_sol_ideal.quick_look()
    output_magma_sol_ideal.to_excel("HHeCNOSi_magma_sol_ideal")

    model_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_magma_sol_real)
    initial_log_number_density = output_magma_sol_ideal.log_number_density
    model_magma_sol_real.solve(
            planet=Uranus,
            mass_constraints=mass_constraints,
            initial_log_number_density=initial_log_number_density,
            )
    output_magma_sol_real = model_magma_sol_real.output
    output_magma_sol_real.quick_look()
    output_magma_sol_real.to_excel("HHeCNOSi_magma_sol_real")

    filename = "HHeCNOSi_magma_sol_real.xlsx"
    
    H2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
    H2O_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
    O2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
    SiH4_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
    SiO_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
    CO2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
    CO_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
    CH4_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
    N2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
    NH3_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
    He_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="He_g")["pressure"]
    Mg_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="Mg_g")["pressure"]
    MgH_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="HMg_g")["pressure"]
    MgOH_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="HMgO_g")["pressure"]
    tot_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

    H_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
    O_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
    Si_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]
    C_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
    He_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
    N_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
    Mg_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_Mg")["atmosphere_moles"]

    H_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["dissolved_moles"]
    O_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["dissolved_moles"]
    Si_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["dissolved_moles"]
    C_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_C")["dissolved_moles"]
    He_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_He")["dissolved_moles"]
    N_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_N")["dissolved_moles"]
    Mg_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_Mg")["dissolved_moles"]

    H_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["total_moles"]
    O_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["total_moles"]
    Si_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["total_moles"]
    C_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_C")["total_moles"]
    He_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_He")["total_moles"]
    N_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_N")["total_moles"]
    Mg_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_Mg")["total_moles"]

    H_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
    O_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
    Si_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]
    C_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
    He_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
    N_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
    Mg_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_Mg")["atmosphere_mass"]

    H_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["dissolved_mass"]
    O_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["dissolved_mass"]
    Si_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["dissolved_mass"]
    C_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_C")["dissolved_mass"]
    He_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_He")["dissolved_mass"]
    N_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_N")["dissolved_mass"]
    Mg_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_Mg")["dissolved_mass"]

    H_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
    O_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
    Si_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]
    C_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
    He_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
    N_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
    Mg_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_Mg")["total_mass"]

    fig, axes = plt.subplots(3, 3, figsize=(10, 8))
    axes = axes.flatten()  # flatten 2D array of axes to 1D list

    axes[0].plot(f_envs, tot_pressure_magma_sol_real, color=color_tot, lw=4, ls="-", label="Total")
    axes[0].plot(f_envs, H2_pressure_magma_sol_real, color=color_H2, lw=3, ls="-", label="H$_2$")
    axes[0].plot(f_envs, H2O_pressure_magma_sol_real, color=color_H2O, lw=3, ls="-", label="H$_2$O")
    axes[0].plot(f_envs, SiH4_pressure_magma_sol_real, color=color_SiH4, lw=3, ls="-", label="SiH$_4$")
    axes[0].plot(f_envs, SiO_pressure_magma_sol_real, color=color_SiO, lw=3, ls="-", label="SiO")
    axes[0].plot(f_envs, CO2_pressure_magma_sol_real, color=color_CO2, lw=4, ls="-", label="CO$_2$")
    axes[0].plot(f_envs, CO_pressure_magma_sol_real, color=color_CO, lw=3, ls="-", label="CO")
    axes[0].plot(f_envs, CH4_pressure_magma_sol_real, color=color_CH4, lw=3, ls="-", label="CH$_4$")
    axes[0].plot(f_envs, N2_pressure_magma_sol_real, color=color_N2, lw=3, ls="-", label="N$_2$")
    axes[0].plot(f_envs, NH3_pressure_magma_sol_real, color=color_NH3, lw=3, ls="-", label="NH$_3$")
    axes[0].plot(f_envs, He_pressure_magma_sol_real, color=color_He, lw=3, ls="-", label="He")
    axes[0].plot(f_envs, Mg_pressure_magma_sol_real, color=color_Mg, lw=3, ls="-", label="Mg")
    axes[0].plot(f_envs, MgH_pressure_magma_sol_real, color=color_MgH, lw=3, ls="-", label="MgH")
    axes[0].plot(f_envs, MgOH_pressure_magma_sol_real, color=color_MgOH, lw=3, ls="-", label="MgOH")
    axes[0].plot(f_envs, tot_pressure_magma_sol_real, color=color_tot, lw=3, ls="-", label="tot")
    # ax0.plot(hmps, O2_pressure_magma_sol_real, color=color_O2, lw=3, ls= '-', label='O$_2$')

    axes[0].set_title(r"(a) Ideal gases with sol")
    axes[0].set_ylim([1e0, 2e5])
    axes[0].set_xscale("log")
    axes[0].set_yscale("log")
    axes[0].set_xlabel(r"Envelope Mass / Planet Mass", fontsize=10)
    axes[0].set_ylabel(r"MEB Partial Pressure [bar]", fontsize=12)
    axes[0].set_xticks([0.01, 0.03, 0.1])
    axes[0].get_xaxis().set_major_formatter(ScalarFormatter())

    axes[0].legend(fontsize=10, loc='best') 
    ax0 = axes[0].twinx()
    ax0.set_yscale("log")
    ax0.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
    ax0.set_ylim([1e-4, 2e1])


    axes[1].plot(f_envs, H_moles_atm_magma_sol_real, color=color_H, lw=4, ls="-", label="H")
    axes[1].plot(f_envs, O_moles_atm_magma_sol_real, color=color_O, lw=3, ls="-", label="O")
    axes[1].plot(f_envs, Si_moles_atm_magma_sol_real, color=color_Si, lw=3, ls="-", label="Si")
    axes[1].plot(f_envs, C_moles_atm_magma_sol_real, color=color_C, lw=4, ls="-", label="C")
    axes[1].plot(f_envs, He_moles_atm_magma_sol_real, color=color_He, lw=3, ls="-", label="He")
    axes[1].plot(f_envs, N_moles_atm_magma_sol_real, color=color_N, lw=3, ls="-", label="N")
    axes[1].plot(f_envs, Mg_moles_atm_magma_sol_real, color=color_Mg, lw=3, ls="-", label="Mg")
    # ax0.plot(hmps, O2_pressure_magma_sol_real, color=color_O2, lw=3, ls= '-', label='O$_2$')

    #axes[1].set_ylim([1e0, 2e5])
    axes[1].set_xscale("log")
    axes[1].set_yscale("log")
    axes[1].set_xlabel(r"Envelope Mass / Planet Mass", fontsize=10)
    axes[1].set_ylabel("Atmosphere moles", fontsize=12)
    axes[1].set_xticks([0.01, 0.03, 0.1])
    axes[1].get_xaxis().set_major_formatter(ScalarFormatter())
    axes[1].legend(fontsize=10, loc='best') 


    axes[2].plot(f_envs, H_moles_melt_magma_sol_real, color=color_H, lw=4, ls="-", label="H")
    axes[2].plot(f_envs, O_moles_melt_magma_sol_real, color=color_O, lw=3, ls="-", label="O")
    axes[2].plot(f_envs, Si_moles_melt_magma_sol_real, color=color_Si, lw=3, ls="-", label="Si")
    axes[2].plot(f_envs, C_moles_melt_magma_sol_real, color=color_C, lw=4, ls="-", label="C")
    axes[2].plot(f_envs, He_moles_melt_magma_sol_real, color=color_He, lw=3, ls="-", label="He")
    axes[2].plot(f_envs, N_moles_melt_magma_sol_real, color=color_N, lw=3, ls="-", label="N")
    axes[2].plot(f_envs, Mg_moles_melt_magma_sol_real, color=color_Mg, lw=3, ls="-", label="Mg")
    # ax0.plot(hmps, O2_pressure_magma_sol_real, color=color_O2, lw=3, ls= '-', label='O$_2$')

    #axes[1].set_ylim([1e0, 2e5])
    axes[2].set_xscale("log")
    axes[2].set_yscale("log")
    axes[2].set_xlabel(r"Envelope Mass / Planet Mass", fontsize=10)
    axes[2].set_ylabel("Dissolved moles", fontsize=12)
    axes[2].set_xticks([0.01, 0.03, 0.1])
    axes[2].get_xaxis().set_major_formatter(ScalarFormatter())
    axes[2].legend(fontsize=10, loc='best') 


    axes[3].plot(f_envs, H_moles_tot_magma_sol_real, color=color_H, lw=4, ls="-", label="H")
    axes[3].plot(f_envs, O_moles_tot_magma_sol_real, color=color_O, lw=3, ls="-", label="O")
    axes[3].plot(f_envs, Si_moles_tot_magma_sol_real, color=color_Si, lw=3, ls="-", label="Si")
    axes[3].plot(f_envs, C_moles_tot_magma_sol_real, color=color_C, lw=4, ls="-", label="C")
    axes[3].plot(f_envs, He_moles_tot_magma_sol_real, color=color_He, lw=3, ls="-", label="He")
    axes[3].plot(f_envs, N_moles_tot_magma_sol_real, color=color_N, lw=3, ls="-", label="N")
    axes[3].plot(f_envs, Mg_moles_tot_magma_sol_real, color=color_Mg, lw=3, ls="-", label="Mg")
    # ax0.plot(hmps, O2_pressure_magma_sol_real, color=color_O2, lw=3, ls= '-', label='O$_2$')

    #axes[1].set_ylim([1e0, 2e5])
    axes[3].set_xscale("log")
    axes[3].set_yscale("log")
    axes[3].set_xlabel(r"Envelope Mass / Planet Mass", fontsize=10)
    axes[3].set_ylabel("Total moles", fontsize=12)
    axes[3].set_xticks([0.01, 0.03, 0.1])
    axes[3].get_xaxis().set_major_formatter(ScalarFormatter())
    axes[3].legend(fontsize=10, loc='best') 


    axes[4].plot(f_envs, H_mass_atm_magma_sol_real, color=color_H, lw=4, ls="-", label="H")
    axes[4].plot(f_envs, O_mass_atm_magma_sol_real, color=color_O, lw=3, ls="-", label="O")
    axes[4].plot(f_envs, Si_mass_atm_magma_sol_real, color=color_Si, lw=3, ls="-", label="Si")
    axes[4].plot(f_envs, C_mass_atm_magma_sol_real, color=color_C, lw=4, ls="-", label="C")
    axes[4].plot(f_envs, He_mass_atm_magma_sol_real, color=color_He, lw=3, ls="-", label="He")
    axes[4].plot(f_envs, N_mass_atm_magma_sol_real, color=color_N, lw=3, ls="-", label="N")
    axes[4].plot(f_envs, Mg_mass_atm_magma_sol_real, color=color_Mg, lw=3, ls="-", label="Mg")
    # ax0.plot(hmps, O2_pressure_magma_sol_real, color=color_O2, lw=3, ls= '-', label='O$_2$')

    #axes[1].set_ylim([1e0, 2e5])
    axes[4].set_xscale("log")
    axes[4].set_yscale("log")
    axes[4].set_xlabel(r"Envelope Mass / Planet Mass", fontsize=10)
    axes[4].set_ylabel("Atmosphere mass", fontsize=12)
    axes[4].set_xticks([0.01, 0.03, 0.1])
    axes[4].get_xaxis().set_major_formatter(ScalarFormatter())
    axes[4].legend(fontsize=10, loc='best') 


    axes[5].plot(f_envs, H_mass_melt_magma_sol_real, color=color_H, lw=4, ls="-", label="H")
    axes[5].plot(f_envs, O_mass_melt_magma_sol_real, color=color_O, lw=3, ls="-", label="O")
    axes[5].plot(f_envs, Si_mass_melt_magma_sol_real, color=color_Si, lw=3, ls="-", label="Si")
    axes[5].plot(f_envs, C_mass_melt_magma_sol_real, color=color_C, lw=4, ls="-", label="C")
    axes[5].plot(f_envs, He_mass_melt_magma_sol_real, color=color_He, lw=3, ls="-", label="He")
    axes[5].plot(f_envs, N_mass_melt_magma_sol_real, color=color_N, lw=3, ls="-", label="N")
    axes[5].plot(f_envs, Mg_mass_melt_magma_sol_real, color=color_Mg, lw=3, ls="-", label="Mg")
    # ax0.plot(hmps, O2_pressure_magma_sol_real, color=color_O2, lw=3, ls= '-', label='O$_2$')

    #axes[1].set_ylim([1e0, 2e5])
    axes[5].set_xscale("log")
    axes[5].set_yscale("log")
    axes[5].set_xlabel(r"Envelope Mass / Planet Mass", fontsize=10)
    axes[5].set_ylabel("Dissolved mass", fontsize=12)
    axes[5].set_xticks([0.01, 0.03, 0.1])
    axes[5].get_xaxis().set_major_formatter(ScalarFormatter())
    axes[5].legend(fontsize=10, loc='best') 


    axes[6].plot(f_envs, H_mass_tot_magma_sol_real, color=color_H, lw=4, ls="-", label="H")
    axes[6].plot(f_envs, O_mass_tot_magma_sol_real, color=color_O, lw=3, ls="-", label="O")
    axes[6].plot(f_envs, Si_mass_tot_magma_sol_real, color=color_Si, lw=3, ls="-", label="Si")
    axes[6].plot(f_envs, C_mass_tot_magma_sol_real, color=color_C, lw=4, ls="-", label="C")
    axes[6].plot(f_envs, He_mass_tot_magma_sol_real, color=color_He, lw=3, ls="-", label="He")
    axes[6].plot(f_envs, N_mass_tot_magma_sol_real, color=color_N, lw=3, ls="-", label="N")
    axes[6].plot(f_envs, Mg_mass_tot_magma_sol_real, color=color_Mg, lw=3, ls="-", label="Mg")
    # ax0.plot(hmps, O2_pressure_magma_sol_real, color=color_O2, lw=3, ls= '-', label='O$_2$')

    #axes[1].set_ylim([1e0, 2e5])
    axes[6].set_xscale("log")
    axes[6].set_yscale("log")
    axes[6].set_xlabel(r"Envelope Mass / Planet Mass", fontsize=10)
    axes[6].set_ylabel("Total mass", fontsize=12)
    axes[6].set_xticks([0.01, 0.03, 0.1])
    axes[6].get_xaxis().set_major_formatter(ScalarFormatter())
    axes[6].legend(fontsize=10, loc='best') 

    # Hide unused subplots
    for j in range(7, 9):
        axes[j].axis("off")

    plt.tight_layout()
    title = f"Uranus Interior: {metall:.0f}× Solar Metallicity, T ≈ {surface_temperature:.0f} K"
    filename = f"./images_rocky/uranus_{metall:.0f}xsolar_{surface_temperature:.0f}K_withadded.png"

    plt.title(title)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

In [8]:
temperature = np.linspace(2000,3000,500)
temperature_repeated = np.repeat(temperature, 50)
print(temperature_repeated)
print(len(temperature_repeated))

[2000. 2000. 2000. ... 3000. 3000. 3000.]
25000


In [9]:
temperature = np.linspace(2000,3000,500)
constraints_repeated = np.tile(temperature, int(np.ceil(25000 / len(temperature))))[:25000]
print(constraints_repeated)
print(len(constraints_repeated))

[2000.              2002.004008016032  2004.0080160320642 ...
 2995.991983967936  2997.995991983968  3000.             ]
25000


In [10]:
T_resolution = 250
H_resolution = 2 
#flatten look for numpy mesh, all condensates seperately
temperature = np.repeat(np.linspace(2000,3000,T_resolution), H_resolution)
hmps = np.tile(np.logspace(-1, 1.7, num=H_resolution), int(np.ceil(T_resolution)))[:T_resolution*H_resolution]  # wt% Hn can go higher for exploration, maybe even 50)
#many possible reasons, like solubility
metall = 30
#Make arrays of everything

h_kgs = hmps / 100 * planet_mass  # kg

si_kg_magma: float = Si_massfraction_palme14 * planet_mass
mg_kg_magma: float = Mg_massfraction_palme14 * planet_mass
o_kg_magma: float = O_massfraction_palme14 * planet_mass

# Lodders et al. (2009) Springer book chapter Table 8
si_kgs_solar = h_kgs * Si_masspercent_lodders09 / H_masspercent_lodders09
mg_kgs_solar = h_kgs * Mg_masspercent_lodders09 / H_masspercent_lodders09
o_kgs_solar = h_kgs * O_masspercent_lodders09 / H_masspercent_lodders09
c_kgs_solar = h_kgs * C_masspercent_lodders09 / H_masspercent_lodders09
n_kgs_solar = h_kgs * N_masspercent_lodders09 / H_masspercent_lodders09
he_kgs_solar = h_kgs * He_masspercent_lodders09 / H_masspercent_lodders09

mass_constraints = {
    "H": h_kgs,
    "He": he_kgs_solar,
    "C": metall * c_kgs_solar,
    "N": metall * n_kgs_solar,
    "Si": metall * si_kgs_solar + mantle_melt_fraction * si_kg_magma,
    "O": metall * o_kgs_solar + mantle_melt_fraction * o_kg_magma,
    "Mg": metall * mg_kgs_solar + mantle_melt_fraction * mg_kg_magma,
        }

print(mass_constraints)
Uranus = Planet(
    surface_temperature=temperature,
    planet_mass=planet_mass,
    mantle_melt_fraction=mantle_melt_fraction,
    surface_radius=surface_radius,
    )

model_magma_nosol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_nosol_ideal)
model_magma_nosol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
         )
output_magma_nosol_ideal = model_magma_nosol_ideal.output
output_magma_nosol_ideal.quick_look()
output_magma_nosol_ideal.to_excel("HHeCNOSi_magma_nosol_ideal")

model_magma_sol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_sol_ideal)
initial_log_number_density = output_magma_nosol_ideal.log_number_density
model_magma_sol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        initial_log_number_density=initial_log_number_density,
        )
output_magma_sol_ideal = model_magma_sol_ideal.output
output_magma_sol_ideal.quick_look()
output_magma_sol_ideal.to_excel("HHeCNOSi_magma_sol_ideal")

model_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_magma_sol_real)
initial_log_number_density = output_magma_sol_ideal.log_number_density
model_magma_sol_real.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        initial_log_number_density=initial_log_number_density,
        )
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel("HHeCNOSi_magma_sol_real")

filename = "HHeCNOSi_magma_sol_real.xlsx"
        
H2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="He_g")["pressure"]
Mg_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="Mg_g")["pressure"]
MgOH_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="HMgO_g")["pressure"]
MgH_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="HMg_g")["pressure"]
tot_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

fig, axes = plt.subplots(1, 1, figsize=(15, 20))

axes.plot(temperature, tot_pressure_magma_sol_real, color=color_tot, lw=4, ls="-", label="Total")
axes.plot(temperature, H2_pressure_magma_sol_real, color=color_H2, lw=3, ls="-", label="H$_2$")
axes.plot(temperature, H2O_pressure_magma_sol_real, color=color_H2O, lw=3, ls="-", label="H$_2$O")
axes.plot(temperature, SiH4_pressure_magma_sol_real, color=color_SiH4, lw=3, ls="-", label="SiH$_4$")
axes.plot(temperature, SiO_pressure_magma_sol_real, color=color_SiO, lw=3, ls="-", label="SiO")
axes.plot(temperature, CO2_pressure_magma_sol_real, color=color_CO2, lw=4, ls="-", label="CO$_2$")
axes.plot(temperature, CO_pressure_magma_sol_real, color=color_CO, lw=3, ls="-", label="CO")
axes.plot(temperature, CH4_pressure_magma_sol_real, color=color_CH4, lw=3, ls="-", label="CH$_4$")
axes.plot(temperature, N2_pressure_magma_sol_real, color=color_N2, lw=3, ls="-", label="N$_2$")
axes.plot(temperature, NH3_pressure_magma_sol_real, color=color_NH3, lw=3, ls="-", label="NH$_3$")
axes.plot(temperature, He_pressure_magma_sol_real, color=color_He, lw=3, ls="-", label="He")
axes.plot(temperature, Mg_pressure_magma_sol_real, color=color_Mg, lw=3, ls="-", label="Mg")
axes.plot(temperature, MgH_pressure_magma_sol_real, color=color_MgH, lw=3, ls="-", label="MgH")
axes.plot(temperature, MgOH_pressure_magma_sol_real, color=color_MgOH, lw=3, ls="-", label="MgOH")
axes.plot(temperature, tot_pressure_magma_sol_real, color=color_tot, lw=3, ls="-", label="tot")

axes.set_title(r"(a) Real gas")
axes.set_ylim([1e-3, 2e5])
axes.set_yscale("log")
axes.set_xlabel(r"Temperature", fontsize=10)
axes.set_ylabel(r"MEB Partial Pressure [bar]", fontsize=12)
axes.get_xaxis().set_major_formatter(ScalarFormatter())

axes.legend(fontsize=10, loc='best') 
        
plt.tight_layout()
title = f"Uranus Interior: {metall:.0f}× Solar Metallicity, H %wt ≈ {perc:.2f} "
filename = f"./images_pressures2/uranus_{metall:.0f}xperc_{perc:.2f}.png"

plt.title(title)
plt.savefig(filename, dpi=300, bbox_inches='tight')
plt.show()

[14:28:02 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, NoSolubility', 'H2_g: IdealGas, NoSolubility', 'O2_g: IdealGas, NoSolubility', 'OSi_g: IdealGas, NoSolubility', 'H4Si_g: IdealGas, NoSolubility', 'O2Si_l: CondensateActivity, NoSolubility', 'CO2_g: IdealGas, NoSolubility', 'CO_g: IdealGas, NoSolubility', 'CH4_g: IdealGas, NoSolubility', 'N2_g: IdealGas, NoSolubility', 'H3N_g: IdealGas, NoSolubility', 'He_g: IdealGas, NoSolubility', 'Mg_g: IdealGas, NoSolubility', 'MgO_l: CondensateActivity, NoSolubility', 'MgO_cr: CondensateActivity, NoSolubility', 'HMg_g: IdealGas, NoSolubility', 'HMgO_g: IdealGas, NoSolubility')
[14:28:02 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 1.0 CO2_g = 1.0 H2O_g + 1.0 CO_g',
 1: '4.0 H2_g + 1.0 CO2_g = 2.0 H2O_g + 1.0 CH4_g',
 2: '4.0 H2_g + 1.0 O2Si_l = 2.0 H2O_g + 1.0 H4Si_g',
 3: '1.5 H2_g + 0.5 N2_g = 1.0 H3N_g',
 4: '2.0 H2O_g = 2.0 H2_g + 1.0 O2_g',
 5: '1.0 H2_g + 1.0 O2Si_l = 1.0 H2O_

{'H': array([8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.350305187884723e+25,
       8.680000000000000e+22, 4.35030518

[14:28:13 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[14:28:13 - atmodeller.classes             - INFO     ] - Solver steps (max) = 59
[14:28:17 - atmodeller.output              - INFO     ] - Writing output to excel
[14:28:17 - atmodeller.output              - INFO     ] - Computing to_dataframes output
[14:28:17 - atmodeller.output              - INFO     ] - Computing asdict output
[14:28:34 - atmodeller.output              - INFO     ] - Output written to HHeCNOSi_magma_nosol_ideal.xlsx
[14:28:34 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, SolubilityPowerLaw', 'H2_g: IdealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'OSi_g: IdealGas, NoSolubility', 'H4Si_g: IdealGas, NoSolubility', 'O2Si_l: CondensateActivity, NoSolubility', 'CO2_g: IdealGas, _CO2_basalt_dixon95', 'CO_g: IdealGas, _CO_basalt_yoshioka19', 'CH4_g: IdealGas, _CH4_basalt_ardia13', 'N2_g: IdealGas, _N2_basalt_libour

EquinoxRuntimeError: Above is the stack outside of JIT. Below is the stack inside of JIT:
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/equinox/_module/_prebuilt.py", line 81, in __call__
    return self.func(*self.args, *args, **kwargs, **self.keywords)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mathias/atmodeller/atmodeller/solvers.py", line 59, in solver_single
    sol: optx.Solution = optx.root_find(
                         ^^^^^^^^^^^^^^^
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/optimistix/_root_find.py", line 220, in root_find
    return iterative_solve(
           ^^^^^^^^^^^^^^^^
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/optimistix/_iterate.py", line 346, in iterative_solve
    ) = adjoint.apply(_iterate, rewrite_fn, inputs, tags)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/equinox/_mo

In [ ]:
output_magma_sol_ideal = model_magma_sol_ideal.output
output_magma_sol_ideal.quick_look()

model_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_magma_sol_real)
initial_log_number_density = output_magma_sol_ideal.log_number_density
model_magma_sol_real.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        initial_log_number_density=initial_log_number_density,
        )
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel("HHeCNOSi_magma_sol_real")